In [153]:
shapes = [


    ['..#',
     '.##',
     '##.'],

    ['###',
     '.##',
     '..#'],

    ['.##',
     '##.',
     '###'],

    ['###',
     '.#.',
     '###'],

     ['###',
      '.##',
      '.##'],

     ['###',
      '#..',
      '###']



]





In [154]:
import re


def read_input(filename):

    # p_figstart = re.compile('[0-9]+:')
    buf = []
    p_area = re.compile('[\d]+x[\d]+:')
    with open(filename) as file:
        
        for line in file:
            if p_area.match(line):
                
                area, counts = line.split(':')

                w,h=[int(v) for v in area.split('x')]
                counts = [int(v) for v in counts.strip().split(' ')]

                # print(w,h, counts)
                # read thre lines of a figure

                buf.append({'w':w, 'h':h, 'counts':counts})


         
    return buf

lines = read_input('./test_input.txt')

print(lines)

[{'w': 4, 'h': 4, 'counts': [0, 0, 0, 0, 2, 0]}, {'w': 12, 'h': 5, 'counts': [1, 0, 1, 0, 2, 2]}, {'w': 12, 'h': 5, 'counts': [1, 0, 1, 0, 3, 2]}]


In [155]:
def count_pounds(shape):
    return sum([ 1 for c in ''.join(shape) if c=='#'])

count_pounds(shapes[2])


7

Figuring out if it is:

-   a definitive  NO, because the number of # exceeds w*h
-   or a definitive YES, because even treating all figs as 3x3 fits

In [156]:
def get_min_space_req(line, shapes):

    return sum([count_pounds(shapes[i]) * c for i, c in enumerate(line['counts'])])


for line in lines:
    area = line['w']*line['h']
    min_req =get_min_space_req(line, shapes)  
    print(line['w'],'x' ,line['h'], ' == ', area, min_req, 'Theoretically fits: ', area>=min_req, end='\t' )

    squares_space = (line['w'] // 3) * (line['h']//3)
    max_req = sum(line['counts']) 
    print('Definetely fits: ', squares_space >= max_req )



4 x 4  ==  16 14 Theoretically fits:  True	Definetely fits:  False
12 x 5  ==  60 40 Theoretically fits:  True	Definetely fits:  False
12 x 5  ==  60 47 Theoretically fits:  True	Definetely fits:  False


# Heuristics baby

In [157]:
a = list(range(10))

n=8
l=5

a = np.array(a)
np.concatenate( [a[0:n],  a[n:n+l] + 10 , a[n+l:]])


array([ 0,  1,  2,  3,  4,  5,  6,  7, 18, 19])

In [158]:
np.repeat([5],10)

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [163]:
import numpy as np
from itertools import product
import matplotlib.pyplot as plt
import re


def read_input(filename):

    buf = []
    p_area = re.compile('[\d]+x[\d]+:')
    with open(filename) as file:
        for line in file:
            if p_area.match(line):
                area, counts = line.split(':')
                w,h=[int(v) for v in area.split('x')]
                counts = [int(v) for v in counts.strip().split(' ')]
                buf.append({'w':w, 'h':h, 'counts':counts})
    return buf


# Combinations
heuristics = [
#   set         space w, h (with variations)
    
    


    [[0, 1, 3], [[7, 3], [3, 7]]],
    [[0, 2, 0], [[7, 3], [3, 7]]],
    [[3, 5, 3], [[7, 3], [3, 7]]],
    [[4,4], [[4,4]]],
    
    [[0, 1], [[5, 3], [3, 5]]],
    [[0, 2], [[5, 3], [3, 5]]],
    [[3, 4], [[5, 3], [3, 5]]],
    [[3, 5], [[5, 3], [3, 5]]],
    [[3, 0], [[5, 3], [3, 5]]],
    

    [[0], [[3,3]]],
    [[1], [[3,3]]],
    [[2], [[3,3]]],
    [[3], [[3,3]]],
    [[4], [[3,3]]],
    [[5], [[3,3]]],
]





def apply_heuristic(counts, heuristic):    
    # pattern = [1 if i in heuristic else 0 for i in range(len(counts))]
    result = counts[:]
    # print('h', heuristic)
    for h in heuristic:
        result[h] -= 1
    return result



# test_heuristic([1,0,1,0,2,2], [0,2,4])


def apply_heuristics(line, heuristics):
    
    new_blocks = []
    
    remains = line['counts'][:]
    # print(new_blocks)
    while sum(remains) > 0:
        for i, h in enumerate(heuristics):
            while True:
                result = apply_heuristic(remains, h[0])
                valid_heuristic = all(np.array(result)>=0)
                if valid_heuristic:
                    # print('applying ', h[0], h[1])
                    new_blocks.append(h[1])
                    remains = result
                else:
                    break
            # print(remains, h, valid, remains)
        # break

    # print('All possibilities!',line['w'],line['h'], new_blocks)
    return new_blocks



def generate_layouts(blocks):

    def increment(state, max_values):
        k = 0
        while k<len(state):
            if state[k]+1 <= max_values[k]:
                state[k] += 1
                return state
            else:
                state[k] = 0
                k += 1
        return None

    l = len(blocks)
    m = [len(p)-1 for p in blocks]
    state = np.zeros(l)
    layouts = []
    while state is not None:

        # print(state)
        layout = [blocks[i][int(s)] for i, s in enumerate(state)]
        layouts.append(layout)

        # does_it_fit = fit(w, h, layout)

        # print(state, layout)
        state = increment(state, m)
        break
    # print(layouts)
    return layouts

   
def fit(w:int,h:int,  layout:list, ax):
    

    def find_optimal_col(block, tops):
        minv= 999999999
        minc = 0
        for i in range(len(tops)-block[0]+1):
            tmp = np.concatenate( [tops[0:i],  np.repeat(max(tops[i:i+block[0]])+block[0], block[0])  , tops[i+block[0]:]])
            # print(i, tmp, max(tmp))
            if max(tmp)<minv:
                minv=max(tmp)
                minc= i
        return minc


    def can_squeeze_block(block, tops, r=0):
        return (max(tops)- max(tops[-block[0]:]) >= block[1]-r )
    

    def find_block_id(block, layout):
        for i, b in enumerate(layout):
            if b == block:
                return i
        return None


    # we do tetris here
    tops = np.zeros(w)
    col = 0
    i = 0
    while len(layout) > 0:
        block = layout[0]

        # ax.plot(np.arange(0, len(tops)), tops, lw=2, color='red')

        col = find_optimal_col(block, tops)

        # checking if there is space for 3x3 a at the end of previous row
        if (col == 0) & (sum(tops)>0):
            has_space_4_3x3 = can_squeeze_block([3,3], tops, 0)
            has_space_4_4x4 = can_squeeze_block([4,4], tops, 1)
            
            if has_space_4_4x4:
                found = find_block_id([4,4], layout)
                if found is not None:
                    layout = [layout[found]] + layout[:found] + layout[found+1:]
                    block = layout[0]
                    col = len(tops)-4

            elif has_space_4_3x3:
                
                # see if there are any 3x3 in the layout
                found = find_block_id([3,3], layout)
                if found is not None:
                    layout = [layout[found]] + layout[:found] + layout[found+1:]
                    block = layout[0]
                    col = len(tops)-3
                    # print('Can squeeze 3x3: ',i,   found) 

                


        if ax is not None:
            ax.add_patch(plt.Rectangle((col,max( tops[col:col + block[0]])), block[0],block[1], ls="--", lw=2,  ec="black", fc="blue", alpha=0.4))
        tops[col:col + block[0]] = max( tops[col:col + block[0]]) + block[1]

        
        if ax is not None:
            ax.text(*(col + block[0]/2 ,max( tops[col:col + block[0]]) - block[1]/2), str(i), ha='center', va='center')



        col = col + block[0]

        layout = layout[1:]
        i += 1


        if any(tops > h):
            # print(f'No space for remaining {len(layout)} blocks')
            return False, len(layout)
        
        
    return True, 0
        



# Read input
# lines = read_input('./test_input.txt')
lines = read_input('./input.txt')
# fig, ax = plt.subplots(1,1, figsize=(10,10))





ax = None
can_fit = []
N = 995

# for i, line in enumerate(lines[N:N+1]):

for i, line in enumerate(lines):
    if i%100 ==0:
        print(i)
    # ax.set_xlim(0,max(line['w'], line['h'])+1)
    # ax.set_ylim(0,max(line['w'], line['h'])+1)
    # ax.add_patch(plt.Rectangle((0,0), line['w'], line['h'], ls="--", lw=2,  ec="black", fc="white"))

    # Apply heuristics to get the opimal block configurations
    new_blocks = apply_heuristics(line, heuristics)
    # print('New blocks:', len(new_blocks))
    # print(new_blocks)

    # given the optimal blocks, generate all their orientations
    # or not due to the complexity
    layouts = generate_layouts(new_blocks)
    # print('layouts:', len(layouts))

  
    for layout in layouts:
        # print('Trying layout: ', layout)
        f, rem = fit(line['w'],line['h'], layout, ax)
        
        if f:
            can_fit.append(i)
            break
        else:
            # print('')
            pass
        break

    # print('line: ', i ,line, f, can_fit)

0
100
200
300
400
500
600
700
800
900


In [165]:
len(can_fit)

528

In [ ]:




apply_heuristics({'w': 35, 'h': 45, 'counts': [39, 40, 37, 39, 39, 48]}, heuristics)



[[[4, 4]],
 [[4, 4]],
 [[4, 4]],
 [[4, 4]],
 [[4, 4]],
 [[4, 4]],
 [[4, 4]],
 [[4, 4]],
 [[4, 4]],
 [[4, 4]],
 [[4, 4]],
 [[4, 4]],
 [[4, 4]],
 [[4, 4]],
 [[4, 4]],
 [[4, 4]],
 [[4, 4]],
 [[4, 4]],
 [[4, 4]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[7, 3], [3, 7]],
 [[3, 3]],
 [[3, 3]],
 [[3, 3]],
 [[3, 3]],
 [[3, 

In [ ]:
a = [1,2,3]
print(a)

a[2], a[1] = a[1], a[2] 
print(a)

[1, 2, 3]
[1, 3, 2]


In [ ]:

        


fit(12,5, [[5, 3], [4, 4], [3, 3]])

Inserting block  [5, 3]  at  0
Inserting block  [4, 4]  at  5
Inserting block  [3, 3]  at  9


True

In [ ]:
a[4:6] = a[5:7]+1
print(a)

[0. 0. 0. 0. 4. 1. 0. 0. 0. 0.]


In [ ]:
def get_min_space_req(line, shapes):

    return sum([count_pounds(shapes[i]) * c for i, c in enumerate(line['counts'])])


for i, line in enumerate(lines):
    area = line['w']*line['h']
    min_req =get_min_space_req(line, shapes)  
    print(i, line['w'],'x' ,line['h'], ' == ', area, min_req, 'Theoretically fits: ', area>=min_req, end='\t' )

 

    squares_space = (line['w'] // 3) * (line['h']//3)
    max_req = sum(line['counts']) 
    print('Definetely fits: ', squares_space >= max_req )


0 50 x 45  ==  2250 1557 Theoretically fits:  True	Definetely fits:  True
1 46 x 50  ==  2300 1560 Theoretically fits:  True	Definetely fits:  True
2 45 x 47  ==  2115 1444 Theoretically fits:  True	Definetely fits:  True
3 50 x 39  ==  1950 1328 Theoretically fits:  True	Definetely fits:  True
4 35 x 45  ==  1575 1576 Theoretically fits:  False	Definetely fits:  False
5 39 x 42  ==  1638 1641 Theoretically fits:  False	Definetely fits:  False
6 36 x 36  ==  1296 1298 Theoretically fits:  False	Definetely fits:  False
7 35 x 47  ==  1645 1648 Theoretically fits:  False	Definetely fits:  False
8 50 x 37  ==  1850 1852 Theoretically fits:  False	Definetely fits:  False
9 35 x 48  ==  1680 1134 Theoretically fits:  True	Definetely fits:  True
10 38 x 48  ==  1824 1825 Theoretically fits:  False	Definetely fits:  False
11 49 x 39  ==  1911 1342 Theoretically fits:  True	Definetely fits:  True
12 35 x 50  ==  1750 1752 Theoretically fits:  False	Definetely fits:  False
13 43 x 41  ==  1763 